### 11.8 RNN/LSTM
論文データを学習させてナンチャッテ論文をlatex形式で出力させるモデルをLSTMで作る

### ライブラリインポート

In [ ]:
!pip install datasets tqdm torch requests feedparser --quiet

In [ ]:
import os
import tarfile
import glob
import re
import requests
from tqdm import tqdm
import urllib.parse
import feedparser
import random

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

### データセット読み込み

#### arXiv API から論文 ID をまとめて取得する関数

In [ ]:
def fetch_arxiv_ids(category="cs.LG", max_results=200):
    """
    指定カテゴリの新しい arXiv 論文 ID をまとめて取得する簡易関数
    例: category="cs.LG"（Machine Learning）
    """
    base_url = "http://export.arxiv.org/api/query"

    # APIのクエリ: カテゴリ指定 + 新しい順
    search_query = f"cat:{category}"
    params = {
        "search_query": search_query,
        "start": 0,
        "max_results": max_results,
        "sortBy": "submittedDate",
        "sortOrder": "descending",
    }

    url = base_url + "?" + urllib.parse.urlencode(params)
    print("🔍 arXiv API URL:", url)

    feed = feedparser.parse(url)

    paper_ids = []
    for entry in feed.entries:
        # entry.id 例: "http://arxiv.org/abs/2401.12345v1"
        m = re.search(r'arxiv.org/abs/(\d{4}\.\d+)', entry.id)
        if m:
            pid = m.group(1)  # "2401.12345"
            paper_ids.append(pid)

    print(f"📄 取得した論文ID数: {len(paper_ids)} 件")
    return paper_ids

# ここで好きなカテゴリ・件数を指定
# 例1: cs.LG (Machine Learning) 最新から200本
# 例2: math.PR (Probability)
paper_ids = fetch_arxiv_ids(category="cs.LG", max_results=200)

print(paper_ids)

#### arXiv ソースをダウンロード＆展開

In [ ]:
os.makedirs("papers", exist_ok=True)

for pid in tqdm(paper_ids, desc="Downloading", ncols=80, ascii=True):
    url = f"https://arxiv.org/src/{pid}v1"
    out_path = f"papers/{pid}.tar.gz"
    try:
        r = requests.get(url, timeout=10)
        if r.status_code == 200:
            with open(out_path, "wb") as f:
                f.write(r.content)
            # tar / tar.gz / その他の圧縮形式に柔軟に対応
            try:
                with tarfile.open(out_path, "r:*") as tar:
                    tar.extractall(f"papers/{pid}")
            except tarfile.ReadError:
                print(f"Skipped {pid} (not a tar file)")
        else:
            print(f"Skipped {pid} (status {r.status_code})")
    except Exception as e:
        print(f"Error downloading {pid}: {e}")


#### 結果確認

In [ ]:
!find papers -name "*.tex" | head -n 20

### データ加工

#### LaTeX 本文抽出 ＋ 前処理

In [ ]:
def extract_latex_body(tex_text):
    """LaTeX本文を抽出してクリーン化"""
    text = re.sub(r'%.*', '', tex_text)  # コメント削除
    m = re.search(r'\\begin{document}(.*?)\\end{document}', text, re.DOTALL)
    if m:
        text = m.group(1)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [ ]:
import re, glob

all_texts = []
for fname in glob.glob("papers/**/*.tex", recursive=True):
    try:
        with open(fname, encoding="utf-8", errors="ignore") as f:
            raw = f.read()
            cleaned = extract_latex_body(raw)  # ← あなたの定義した関数
            if len(cleaned) > 200:
                all_texts.append(cleaned)
                # print(f"抽出成功: {fname}")
    except Exception as e:
        print(f"{fname}: {e}")

print("抽出された論文本数:", len(all_texts))

In [ ]:
# all_texts からトークンを作るところ（←これが先）

TOKEN_PATTERN = r'\\[a-zA-Z]+|[{}_^=+\-\*/()0-9a-zA-Z]+'
SPECIAL_TOKENS = ["<unk>", "<bos>", "<eos>"]

tokens = []
for txt in all_texts:
    # 各論文ごとに <bos>, <eos> を挟んでトークナイズ
    body_tokens = re.findall(TOKEN_PATTERN, txt)
    tokens.extend(["<bos>"] + body_tokens + ["<eos>"])

print(f"トークン総数: {len(tokens):,}")

In [ ]:
def extract_latex_body(tex_text: str) -> str:
    """LaTeX本文を抽出してある程度クリーンアップする（簡易版）"""
    # コメント行を削除（% 以降）
    text = re.sub(r'%.*', '', tex_text)

    # \begin{document}〜\end{document} の間だけを抜き出し
    m = re.search(r'\\begin{document}(.*?)\\end{document}', text, re.DOTALL)
    if m:
        text = m.group(1)

    # bibliography / 参考文献・付録以降をざっくり削る（必要に応じて調整）
    text = re.split(r'\\bibliography|\\begin{thebibliography}', text)[0]
    text = re.split(r'\\appendix', text)[0]

    # 連続する空白を1つにまとめる
    text = re.sub(r'\s+', ' ', text).strip()
    return text

all_texts = []
for fname in glob.glob("papers/**/*.tex", recursive=True):
    try:
        with open(fname, encoding="utf-8", errors="ignore") as f:
            raw = f.read()
        cleaned = extract_latex_body(raw)
        if len(cleaned) > 200:  # ごく短いファイルはノイズなので除外
            all_texts.append(cleaned)
            #print(f"xtracted from: {fname}")
    except Exception as e:
        print(f"{fname}: {e}")

print(f"\n抽出された論文本数: {len(all_texts)}")
print(f"総テキスト長: {sum(len(t) for t in all_texts):,} 文字")

# テキストを一つにまとめたファイルも保存しておく（デバッグ用）
merged_text = "\n\n".join(all_texts)
with open("merged_corpus.tex", "w", encoding="utf-8") as f:
    f.write(merged_text)
print("\nmerged_corpus.tex に保存しました")

#### トークン化と語彙辞書作成

In [ ]:
from collections import Counter

# SPECIAL_TOKENS はさっきと同じものを使う
# SPECIAL_TOKENS = ["<unk>", "<bos>", "<eos>"]

print(f"トークン総数: {len(tokens):,}")

# 語彙を頻度上位だけに圧縮
counter = Counter(tokens)

VOCAB_LIMIT = 40000
most_common_tokens = [
    w for (w, c) in counter.most_common(VOCAB_LIMIT)
    if w not in SPECIAL_TOKENS
]

vocab = SPECIAL_TOKENS + most_common_tokens

word2idx = {w: i for i, w in enumerate(vocab)}
idx2word = {i: w for i, w in enumerate(vocab)}

UNK_IDX = word2idx["<unk>"]
BOS_IDX = word2idx["<bos>"]
EOS_IDX = word2idx["<eos>"]

print(f"語彙数（圧縮後）: {len(vocab):,}")

# トークン列を index に変換
indexed_tokens = [word2idx.get(t, UNK_IDX) for t in tokens]

# 念のためチェック
max_id = max(indexed_tokens)
print("最大トークンID:", max_id)
print("語彙サイズ:", len(vocab))
assert max_id < len(vocab), "トークンIDが語彙サイズを超えています！"


### データセット定義

#### Datasetクラス定義

In [ ]:
class LatexDataset(Dataset):
    """大きなトークン列から、ランダムにサンプルを切り出す Dataset"""
    def __init__(self, data, seq_length=60, samples_per_epoch=3000):
        self.data = data                      # ← すでに index 化された列
        self.seq_length = seq_length
        self.samples_per_epoch = samples_per_epoch

        self.max_start = len(self.data) - (self.seq_length + 1)
        assert self.max_start > 0, "データが短すぎます"

    def __len__(self):
        # 1epochあたりのサンプル数を直接決める
        return self.samples_per_epoch

    def __getitem__(self, idx):
        # idx は使わず、毎回ランダムな位置から切り出す
        start = random.randint(0, self.max_start)
        end = start + self.seq_length + 1
        chunk = self.data[start:end]

        x = torch.tensor(chunk[:-1], dtype=torch.long)  # [T]
        y = torch.tensor(chunk[1:], dtype=torch.long)   # [T]
        return x, y


In [ ]:
# ここで初めて dataset を作る（tokens ではなく indexed_tokens を渡す）
seq_length = 60
samples_per_epoch = 8000
dataset = LatexDataset(indexed_tokens, seq_length=seq_length,
                       samples_per_epoch=samples_per_epoch)


#### Train / Validation に分割して DataLoader を作成

In [ ]:
train_ratio = 0.9
n_total = len(dataset)
n_train = int(n_total * train_ratio)
n_val = n_total - n_train

train_ds, val_ds = random_split(dataset, [n_train, n_val])

batch_size = 32

train_loader = DataLoader(
    train_ds,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=2,
    pin_memory=True
)

val_loader = DataLoader(
    val_ds,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    num_workers=2,
    pin_memory=True
)

print(f"データ数: total={n_total}, train={len(train_ds)}, val={len(val_ds)}")
print(f"1epoch のステップ数（train）: {len(train_loader)}")


### 学習

#### LSTMモデル定義(疑似数学論文生成)

In [ ]:
# LSTMモデル定義(疑似数学論文生成)
class LatexTransformer(nn.Module):
    """
    LaTeX トークン列用のシンプルな Transformer 言語モデル
    入力: x [B, T]  (トークンID)
    出力: logits [B, T, V]  (各位置ごとの単語分布のロジット)
    """
    def __init__(self,
                 vocab_size: int,
                 d_model: int = 256,          # 埋め込み＆隠れ次元
                 nhead: int = 4,              # マルチヘッド注意のヘッド数
                 num_layers: int = 4,         # Transformer 層数
                 dim_feedforward: int = 512,  # FFN の中間次元
                 dropout: float = 0.1,
                 max_seq_len: int = 512):
        super().__init__()

        self.vocab_size = vocab_size
        self.d_model = d_model
        self.max_seq_len = max_seq_len

        # 語彙埋め込み
        self.token_embedding = nn.Embedding(vocab_size, d_model)

        # 位置埋め込み（シンプルな learnable embedding）
        self.pos_embedding = nn.Embedding(max_seq_len, d_model)

        # Transformer Encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True  # [B, T, E] で処理
        )
        self.transformer = nn.TransformerEncoder(
            encoder_layer,
            num_layers=num_layers
        )

        # 出力層
        self.fc_out = nn.Linear(d_model, vocab_size)

        # Causal mask をキャッシュ用に保持
        self.register_buffer("mask", None, persistent=False)

    def _generate_square_subsequent_mask(self, sz: int, device: torch.device):
        """
        自己回帰用のマスク (未来のトークンを見ないようにする)
        shape: [T, T]
        """
        if (self.mask is None) or (self.mask.size(0) != sz):
            mask = torch.full((sz, sz), float("-inf"), device=device)
            mask = torch.triu(mask, diagonal=1)
            self.mask = mask
        return self.mask

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        x: [B, T]  トークンID列
        戻り値:
            logits: [B, T, V]
        """
        B, T = x.size()
        device = x.device

        # 位置ID [0, 1, ..., T-1]
        positions = torch.arange(T, device=device).unsqueeze(0).expand(B, T)  # [B, T]

        tok_emb = self.token_embedding(x)          # [B, T, d_model]
        pos_emb = self.pos_embedding(positions)    # [B, T, d_model]
        h = tok_emb + pos_emb                      # [B, T, d_model]

        # causal mask (未来の情報を見ない)
        src_mask = self._generate_square_subsequent_mask(T, device=device)  # [T, T]

        # TransformerEncoder に通す
        # batch_first=True なので h: [B, T, d_model], mask は [T, T]
        h = self.transformer(h, mask=src_mask)     # [B, T, d_model]

        logits = self.fc_out(h)                    # [B, T, V]
        return logits

#### 繰り返し計算

In [ ]:
%%time
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")

vocab_size = len(vocab)
print("vocab_size:", vocab_size)

model = LatexLSTM(vocab_size=vocab_size).to(device)

print("model.fc.out_features:", model.fc.out_features)
assert model.fc.out_features == vocab_size, "モデルの出力次元と語彙数が一致していません"

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

num_epochs = 30
max_grad_norm = 1.0  # 勾配クリッピングの最大ノルム

for epoch in range(num_epochs):
    # -------------------------------
    # Train
    # -------------------------------
    model.train()
    total_train_loss = 0.0

    for x, y in train_loader:
        # x, y: [B, T]
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        logits = model(x)  # [B, T, V]

        # CrossEntropyLoss は [N, C] vs [N] をとるので reshape する
        loss = criterion(
            logits.view(-1, logits.size(-1)),  # [B*T, V]
            y.view(-1)                         # [B*T]
        )
        loss.backward()

        # 勾配クリッピング（勾配爆発対策）
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        optimizer.step()
        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)

    # -------------------------------
    # Validation
    # -------------------------------
    model.eval()
    total_val_loss = 0.0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            logits = model(x)
            val_loss = criterion(
                logits.view(-1, logits.size(-1)),
                y.view(-1)
            )
            total_val_loss += val_loss.item()

    avg_val_loss = total_val_loss / len(val_loader)

    # 学習率スケジューラを進める
    scheduler.step()

    print(
        f"Epoch [{epoch+1}/{num_epochs}] "
        f"train_loss={avg_train_loss:.4f}  val_loss={avg_val_loss:.4f}  "
        f"lr={scheduler.get_last_lr()[0]:.5f}"
    )


### 予測

#### サンプリング用ヘルパー




In [ ]:
def sample_from_logits(logits: torch.Tensor,
                       temperature: float = 0.8,
                       top_k: int = 50) -> int:
    """
    logits: [V] の1時刻分のロジット
    戻り値: 次に選ぶトークンID
    """
    # 温度でスケーリング
    logits = logits / temperature

    # top-k だけ残してサンプリング（極端なノイズを減らす）
    if top_k is not None and top_k > 0:
        values, indices = torch.topk(logits, top_k)
        probs = torch.softmax(values, dim=-1)
        idx_in_topk = torch.multinomial(probs, 1).item()
        next_token_id = indices[idx_in_topk].item()
        return next_token_id
    else:
        probs = torch.softmax(logits, dim=-1)
        next_token_id = torch.multinomial(probs, 1).item()
        return next_token_id


#### テキスト生成関数

In [ ]:
def generate(seed_text, length=200, temperature=0.8, top_k=50, seq_length=60):
    model.eval()
    words = re.findall(TOKEN_PATTERN, seed_text)

    for _ in range(length):
        # 🔥 直近 seq_length 分だけ使う
        seq = [word2idx.get(w, UNK_IDX) for w in words[-seq_length:]]

        x = torch.tensor([seq], dtype=torch.long).to(device)

        with torch.no_grad():
            logits = model(x)[0, -1]  # ラスト時間ステップのみ

            # <unk> を出にくくする
            logits[UNK_IDX] -= 10.0

            logits = logits / temperature

            # top-k サンプリング（推奨）
            values, indices = torch.topk(logits, top_k)
            probs = torch.softmax(values, dim=0)
            next_idx = indices[torch.multinomial(probs, 1).item()].item()

        words.append(idx2word[next_idx])

    return " ".join(words)


#### テキスト生成サンプル

In [ ]:
seed = r"\section{Introduction} We consider the problem of minimizing"
print("\n生成結果:\n")
text = generate(seed, length=200, temperature=0.7, top_k=30, seq_length=seq_length)
print(text)

In [ ]:
# =====================================================
# LaTeX → PDF 変換 (Google Colab / Linux 環境用)
# =====================================================

import os, subprocess

# ① 生成済みLaTeX本文をここに貼る（整形後のtextを入れる）
latex_code = r"""
\documentclass[11pt]{article}

% 必要最低限のパッケージ
\usepackage{amsmath, amssymb, amsfonts}
\usepackage{bm}
\usepackage[margin=25mm]{geometry}

\begin{document}

\section{Introduction}

We consider the problem of minimizing an approximation function under
a general learning framework. In this work, we introduce a unified model
that aims to improve prediction performance and enhance optimization stability
across a variety of datasets.

To study the theoretical properties of our model, we begin by examining
a key lemma related to empirical risk minimization. A simplified form
of the proposed inequality can be written as
\begin{equation}
    \mathcal{A}_{\text{train}}(x)
    \leq \mathcal{A}_{\text{test}}(x) + \epsilon,
\end{equation}
where $\epsilon$ is a small constant representing the empirical gap between
training and test distributions.

Furthermore, we consider the iterative update process of the model parameters.
For example, the next state $\mathbf{x}_{t+1}$ may be approximated by
\begin{equation}
    \mathbf{x}_{t+1}
    = \mathbf{x}_{t} - \eta \, \nabla_{\mathbf{x}} \mathcal{L}(\mathbf{x}_t),
\end{equation}
where $\eta$ is a learning rate and $\mathcal{L}$ denotes the loss function.
This provides a generic template for gradient-based optimization in our framework.

The main contributions of this paper can be summarized as follows:
\begin{itemize}
    \item We propose a unified learning framework applicable to various tasks.
    \item We provide theoretical insights into the stability of the proposed model.
    \item We demonstrate competitive performance across multiple datasets.
\end{itemize}

\section{Conclusion}

The generated LaTeX text has been reformatted so that it can be compiled
without errors. Although the original automatically generated expressions
were incomplete, we replaced them with valid representative examples
to ensure successful PDF generation.

\end{document}
"""

# ② texファイルとして保存
with open("generated.tex", "w", encoding="utf-8") as f:
    f.write(latex_code)

# ③ LaTeXコマンドが使えるようにTexLiveをインストール
!apt-get update -qq
!apt-get install -y texlive-latex-base texlive-latex-extra texlive-fonts-recommended > /dev/null

# ④ pdflatexでPDFに変換
!pdflatex -interaction=nonstopmode generated.tex > /dev/null

# ⑤ 出力確認
print("PDF生成完了: generated.pdf")
!ls -lh generated.pdf
